<a href="https://colab.research.google.com/github/jwb4335/nlp/blob/master/hw2_part1/hw2part1_johnbarry_20200120.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework 2 part 1
## CS 590.07
## John Barry





## Load in simlex999 data from github

In [0]:
import pandas as pd
from random import randint



In [0]:
url = 'https://raw.githubusercontent.com/jwb4335/nlp/master/hw2_part1/Sim_Lex999.txt'
simlex = pd.read_csv(url,sep = ':')
simlex.columns = ['word1','word2','SimLex999','POS1','POS2']



```
# This is formatted as code
```

Let's get three random rows

In [5]:
randints = [randint(0,len(simlex)), randint(0,len(simlex)),randint(0,len(simlex))]

simlex_rand = simlex.iloc[randints,:]
simlex_rand



,word1,word2,SimLex999,POS1,POS2
213,grass,blade,4.57,n,n
186,victory,triumph,8.98,n,n
984,cooperate,join,5.18,v,v


## **Universal sentence encoder** 





In [0]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet seaborn

In [7]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [9]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Compute a representation for each message, showing various lengths supported.
word = "breakfast"
messages = [word]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: breakfast
Embedding size: 512
Embedding: [-0.016404513269662857, -0.04483848437666893, -0.0037806909531354904, ...]



In [10]:
message_embedding_snippet

'-0.016404513269662857, -0.04483848437666893, -0.0037806909531354904'

In [0]:
word = "breakfast"
messages = [word]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

In [12]:
message_embeddings

array([[-1.64045133e-02, -4.48384844e-02, -3.78069095e-03,
         4.68277894e-02, -7.71008851e-03, -5.05571701e-02,
        -6.78127445e-03, -3.27876322e-02, -6.28904551e-02,
        -4.78958301e-02,  5.76800331e-02, -5.30807972e-02,
         2.26386786e-02,  2.00913250e-02, -1.91726331e-02,
         1.45138241e-02, -9.16321948e-03, -3.39176925e-03,
        -2.80498969e-03, -3.79767418e-02,  4.18685153e-02,
        -6.24692850e-02, -2.87507810e-02, -3.40368375e-02,
        -3.47199216e-02, -2.53357105e-02, -4.59449776e-02,
        -7.09898174e-02, -6.53066039e-02,  3.80705930e-02,
        -3.92757617e-02, -2.31163409e-02, -3.94491926e-02,
         1.65286213e-02,  3.73129509e-02, -5.67618310e-02,
        -3.04891113e-02,  5.97833432e-02, -2.56873239e-02,
        -6.43153442e-04,  5.93997464e-02,  5.53347953e-02,
         2.13192087e-02, -5.59708215e-02,  4.60403711e-02,
         2.85874866e-02,  3.25253457e-02, -7.22681209e-02,
         5.07617891e-02, -1.03512416e-02, -5.29682003e-0

# CBOW or word2index

Download the data

In [13]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/reviews.txt"
!cat reviews.txt | head

--2020-01-26 00:52:43--  https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33678267 (32M) [text/plain]
Saving to: ‘reviews.txt.2’

reviews.txt.2       100%[===================>]  32.12M   160MB/s    in 0.2s    

2020-01-26 00:52:44 (160 MB/s) - ‘reviews.txt.2’ saved [33678267/33678267]

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the

In [14]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/labels.txt"
!cat labels.txt | head -15

--2020-01-26 00:52:48--  https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt.2’

labels.txt.2        100%[===================>] 219.73K  --.-KB/s    in 0.01s   

2020-01-26 00:52:48 (14.7 MB/s) - ‘labels.txt.2’ saved [225000/225000]

positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive


Predicting Movie Reviews

In [0]:
import sys

f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open('labels.txt')
raw_labels = f.readlines()
f.close()

tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [16]:
vocab[11]

'strangelove'

In [17]:
vocab

['poorest',
 'ravaging',
 'tearoom',
 'profuse',
 'volley',
 'dreamscapes',
 'biker',
 'roughed',
 'pruner',
 'unco',
 'iqs',
 'strangelove',
 'berserker',
 'pfeh',
 'supposes',
 'steenburgen',
 'vilma',
 'debased',
 'verne',
 'arnies',
 'rosenstein',
 'shuffling',
 'senator',
 'atavachron',
 'inexhaustible',
 'vinod',
 'unreported',
 'leviathan',
 'vaunted',
 'zdenek',
 'reigned',
 'nightsky',
 'sania',
 'calico',
 'personify',
 'uecker',
 'lovecraftian',
 'schmaltzy',
 'bugsy',
 'fantasised',
 'recommanded',
 'fukuoka',
 'imitators',
 'soundstages',
 'especial',
 'kane',
 'prenez',
 'actorstallone',
 'adoree',
 'charactershe',
 'rooks',
 'reversion',
 'fraulein',
 'belknap',
 'manhattanites',
 'comes',
 'isaach',
 'antwones',
 'uncover',
 'curious',
 'cyhper',
 'marquee',
 'supreme',
 'neuromuscular',
 'lizards',
 'parfait',
 'glug',
 'humble',
 'flutes',
 'instructional',
 'sandbag',
 'calais',
 'scuffles',
 'classroom',
 'increases',
 'sawtooth',
 'lyons',
 'arbus',
 'rollering',
 

Initialize Weights

In [18]:
import numpy as np
np.random.seed(1)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

alpha, iterations = (0.01, 2)
hidden_size = 100

weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,1)) - 0.1


print (weights_0_1)

[[-0.0165956   0.0440649  -0.09997713 ...  0.0147359  -0.09942593
   0.02342898]
 [-0.03467102  0.00541162  0.07718842 ... -0.05312758  0.02335567
   0.08980326]
 [ 0.09003522  0.01133064  0.08312127 ... -0.08407478  0.09656342
  -0.06367743]
 ...
 [-0.07656205 -0.09023205 -0.00978559 ... -0.0008865  -0.04352412
  -0.06427378]
 [-0.08444666 -0.07143386 -0.05731608 ... -0.06584719 -0.02770722
   0.09327983]
 [ 0.03695404 -0.05106593  0.02970002 ...  0.0043325   0.00801544
  -0.09468272]]


Run the Model

In [19]:


correct,total = (0,0)

for iter in range(iterations):
    
    # train on first 24,000
    for i in range(len(input_dataset)-1000):

        x,y = (input_dataset[i],target_dataset[i])
        layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0)) #embed + sigmoid
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2)) # linear + softmax

        layer_2_delta = layer_2 - y # compare pred with truth
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) #backprop

        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha

        if(np.abs(layer_2_delta) < 0.5):
            correct += 1
        total += 1
        if(i % 10 == 9):
            progress = str(i/float(len(input_dataset)))
            sys.stdout.write('\rIter:'+str(iter)\
                             +' Progress:'+progress[2:4]\
                             +'.'+progress[4:6]\
                             +'% Training Accuracy:'\
                             + str(correct/float(total)) + '%')
    print()
    
correct,total = (0,0)
for i in range(len(input_dataset)-1000,len(input_dataset)):

    x = input_dataset[i]
    y = target_dataset[i]

    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
    
    if(np.abs(layer_2 - y) < 0.5):
        correct += 1
    total += 1
print("Test Accuracy:" + str(correct / float(total)))




Iter:0 Progress:95.99% Training Accuracy:0.8322083333333333%
Iter:1 Progress:95.99% Training Accuracy:0.8660416666666667%
Test Accuracy:0.846


Let's get the vector for a word

In [20]:
target = "dentist"
target_index = word2index[target]
weights_0_1[target_index]

array([-1.48807281e-02, -1.03732417e-01, -7.83948593e-02, -5.60983261e-03,
        6.90961899e-02, -1.87634092e-02,  2.32714624e-02, -7.46111367e-03,
       -1.93403800e-02, -1.16957130e-01,  3.53251328e-02, -2.40042010e-02,
       -6.59379171e-02, -5.30794939e-02, -5.70768620e-02, -6.19464967e-04,
        5.56077696e-02,  1.90993888e-02, -4.66583843e-02, -1.26999956e-01,
       -6.79291853e-02,  7.02844256e-02,  1.14145685e-01,  7.80468502e-02,
        9.37427439e-02,  4.31695064e-02,  1.32140430e-02, -8.75516545e-02,
        3.29443728e-02,  2.69484805e-02, -9.35706289e-02, -8.25100962e-02,
       -1.86231546e-02,  1.53465078e-02,  6.41251715e-02,  3.86167074e-02,
        6.55388290e-02, -7.36236161e-02,  1.46423277e-02,  9.57727123e-02,
        9.66366842e-02,  5.85240210e-02, -9.53830572e-03, -3.09321744e-02,
       -1.34257718e-02, -5.87267310e-02,  7.73963024e-02, -8.60349428e-02,
        2.99778441e-02, -2.10647446e-02, -1.00002047e-04,  7.07221768e-02,
       -6.98387977e-02,  

# Word2vec

In [21]:
!wget http://mattmahoney.net/dc/text8.zip -O text8.zip

--2020-01-26 00:53:09--  http://mattmahoney.net/dc/text8.zip
Resolving mattmahoney.net (mattmahoney.net)... 67.195.197.75
Connecting to mattmahoney.net (mattmahoney.net)|67.195.197.75|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31344016 (30M) [application/zip]
Saving to: ‘text8.zip’

text8.zip            98%[==================> ]  29.33M   672KB/s    eta 2s     ^C


In [22]:
!unzip text8.zip

Archive:  text8.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of text8.zip or
        text8.zip.zip, and cannot find text8.zip.ZIP, period.


In [23]:
!pip install gensim  

In [0]:
from gensim.models import word2vec

In [0]:
# https://github.com/dav/word2vec/blob/master/scripts/create-text8-vector-data.sh#L16
model = word2vec.Word2Vec(corpus_file='text8',size=100, window=5, min_count=1, workers=4)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Let's get the vector for a word

In [0]:
vector = model.wv['computer']
print(vector)

## Get the similarity scores of the three embeddings

In [0]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
pd.options.mode.chained_assignment = None  # default='warn'
for num in randints:
  word1 = simlex_rand.loc[simlex_rand.index == num]['word1'].values[0]
  word2 = simlex_rand.loc[simlex_rand.index == num]['word2'].values[0]
  print('Embedding this pair from Simlex_999: "{}" and "{}"'.format(word1, word2))
  print('Running universal sentence encoder on "{}"'.format(word1))
  messages = [word1]
  # Reduce logging output.
  tf.logging.set_verbosity(tf.logging.ERROR)

  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(messages))

    for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
      print("Message: {}".format(messages[i]))
      print("Embedding size: {}".format(len(message_embedding)))
      message_embedding_snippet = ", ".join(
          (str(x) for x in message_embedding[:3]))
      print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

  vector1 = message_embedding
  vector1 = np.asarray(vector1)
  vector1 = vector1.reshape(1,512)

  print('Running universal sentence encoder on "{}"'.format(word2))
  messages = [word2]

  # Reduce logging output.
  tf.logging.set_verbosity(tf.logging.ERROR)

  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))

  vector2 = message_embedding
  vector2 = np.asarray(vector2)
  vector2 = vector2.reshape(1,512)

  cos_sim = cosine_similarity(vector1, vector2)
  print('Cosine similarity from Universal sentence encoder for {} and {} = {}'.format(word1,word2,cos_sim) )
  simlex_rand.loc[simlex_rand.index == num,'USE Cosine Sim'] = cos_sim

Embedding method 2

In [0]:
for num in randints:
  word1 = simlex_rand.loc[simlex_rand.index == num]['word1'].values[0]
  word2 = simlex_rand.loc[simlex_rand.index == num]['word2'].values[0]
  print('running word2index on the pair {} and {}'.format(word1,word2))
  target_index1 = word2index[word1]
  target_index2 = word2index[word2]
  vector1 = weights_0_1[target_index1]
  vector1 = vector1.reshape(1,100)
  vector2 = weights_0_1[target_index2]
  vector2 = vector2.reshape(1,100)
  cos_sim = cosine_similarity(vector1, vector2)
  print('Cosine similarity from word2index for {} and {} = {}'.format(word1,word2,cos_sim))
  simlex_rand.loc[simlex_rand.index == num,'word2index Cosine Sim'] = cos_sim


Embedding method word2vec



In [0]:
for num in randints:
  word1 = simlex_rand.loc[simlex_rand.index == num]['word1'].values[0]
  word2 = simlex_rand.loc[simlex_rand.index == num]['word2'].values[0]
  print('running word2vec on the pair {} and {}'.format(word1,word2))
  vector1 = model.wv[word1]
  vector1 = vector1.reshape(1,100)
  vector2 = model.wv[word2]
  vector2 = vector2.reshape(1,100)
  cos_sim = cosine_similarity(vector1, vector2)
  print('Cosine similarity from word2vec for {} and {} = {}'.format(word1,word2,cos_sim))
  simlex_rand.loc[simlex_rand.index == num,'word2vec Cosine Sim'] = cos_sim



In [0]:
simlex_rand_out = simlex_rand[['word1','word2','SimLex999', 'USE Cosine Sim',
       'word2index Cosine Sim', 'word2vec Cosine Sim']]
simlex_rand_out.columns = ['word1','word2','SimLex999', 'USE',
       'CBOW', 'word2vec']
simlex_rand_out = round(simlex_rand_out,4)
print('The cosine similarities from the 3 word embedding methods can be seen below.\nThe similarity scores are not close to the Simlex999 scores')
print(simlex_rand_out)

# Part 2, generate three .txt files with similarity scores of all words ending in 4, using my three embedded models.

In [0]:
## Unique id is jwb74, but python starts on zero, so I need to minus 1 to get the right words
unique_id = 4 
index = np.arange(4-1,1000,10)
simlex_out = simlex.iloc[index,:].reset_index(drop=True)[['word1','word2','SimLex999']]
simlex_out.columns = ['word1','word2','Sim_lex_Similarity']
simlex_out

## Embedding method 1

In [0]:
import copy
simlex_out_USE = copy.deepcopy(simlex_out)
for num in np.arange(0,100,1):
  word1 = simlex_out.loc[simlex_out.index == num]['word1'].values[0]
  word2 = simlex_out.loc[simlex_out.index == num]['word2'].values[0]
  try:
    print('Embedding this pair from Simlex_999: "{}" and "{}"'.format(word1, word2))
    print('Running universal sentence encoder on "{}"'.format(word1))
    messages = [word1]
    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)

    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(messages))

      for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
        print("Message: {}".format(messages[i]))
        print("Embedding size: {}".format(len(message_embedding)))
        message_embedding_snippet = ", ".join(
            (str(x) for x in message_embedding[:3]))
        print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

    vector1 = message_embedding
    vector1 = np.asarray(vector1)
    vector1 = vector1.reshape(1,512)

    print('Running universal sentence encoder on "{}"'.format(word2))
    messages = [word2]

    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)

    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(messages))

    for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
      print("Message: {}".format(messages[i]))
      print("Embedding size: {}".format(len(message_embedding)))
      message_embedding_snippet = ", ".join(
          (str(x) for x in message_embedding[:3]))

    vector2 = message_embedding
    vector2 = np.asarray(vector2)
    vector2 = vector2.reshape(1,512)

    cos_sim = cosine_similarity(vector1, vector2)
    print('Cosine similarity from Universal sentence encoder for {} and {} = {}'.format(word1,word2,cos_sim) )
    simlex_out_USE.loc[simlex_out_USE.index == num,'your_Similarity'] = cos_sim
  except:
    print('skipping the pair {} and {}'.format(word1,word2))


## Embedding method 2

In [0]:
simlex_out_CBOW = copy.deepcopy(simlex_out)
for num in np.arange(0,100,1):
  word1 = simlex_out.loc[simlex_out.index == num]['word1'].values[0]
  word2 = simlex_out.loc[simlex_out.index == num]['word2'].values[0]
  #print('running word2index on the pair {} and {}'.format(word1,word2))
  try:
    target_index1 = word2index[word1]
    target_index2 = word2index[word2]
    vector1 = weights_0_1[target_index1]
    vector1 = vector1.reshape(1,100)
    vector2 = weights_0_1[target_index2]
    vector2 = vector2.reshape(1,100)
    cos_sim = cosine_similarity(vector1, vector2)
    #print('Cosine similarity from word2index for {} and {} = {}'.format(word1,word2,cos_sim))
    simlex_out_CBOW.loc[simlex_out_CBOW.index == num,'your_Similarity'] = cos_sim
  except:
    #print('skipping the pair {} and {}'.format(word1,word2))


In [0]:
simlex_out_word2vec = copy.deepcopy(simlex_out)
for num in np.arange(0,100,1):
  word1 = simlex_out.loc[simlex_out.index == num]['word1'].values[0]
  word2 = simlex_out.loc[simlex_out.index == num]['word2'].values[0]
  try:
    #print('running word2vec on the pair {} and {}'.format(word1,word2))
    vector1 = model.wv[word1]
    vector1 = vector1.reshape(1,100)
    vector2 = model.wv[word2]
    vector2 = vector2.reshape(1,100)
    cos_sim = cosine_similarity(vector1, vector2)
    print('Cosine similarity from word2vec for {} and {} = {}'.format(word1,word2,cos_sim))
    simlex_out_word2vec.loc[simlex_out_word2vec.index == num,'your_Similarity'] = cos_sim
  except:
    #print('skipping the pair {} and {}'.format(word1,word2))



In [0]:
from google.colab import files
simlex_out_USE.to_csv('universal_sentence_encoder.csv',sep='\t')
simlex_out_CBOW.to_csv('CBOW.csv', sep='\t')
simlex_out_word2vec.to_csv('word2vec.csv',sep='\t')
files.download('universal_sentence_encoder.csv')
files.download('CBOW.csv')
files.download('word2vec.csv')
